<a href="https://colab.research.google.com/github/JeongeunAhn/edgedetection_MNIST/blob/master/MNIST_edge_detection_double_layer_CNN_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**[CNN-01]** 필요한 모듈을 임포트하고 난수의 시드를 설정한다.

In [0]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import time
from tensorflow.examples.tutorials.mnist import input_data

np.random.seed(20160704)
tf.set_random_seed(20160704)

**[CNN-02]** MNIST 데이터 세트를 준비한다.

In [0]:
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


[edge detector]필터 정보를 저장한 다차원 리스트를 만드는 함수를 준비한다.

In [0]:
def edge_filter():
    filter0 = np.array(
            [[ 2, 1, 0,-1,-2],
             [ 3, 2, 0,-2,-3],
             [ 4, 3, 0,-3,-4],
             [ 3, 2, 0,-2,-3],
             [ 2, 1, 0,-1,-2]]) / 23.0
    filter1 = np.array(
            [[ 2, 3, 4, 3, 2],
             [ 1, 2, 3, 2, 1],
             [ 0, 0, 0, 0, 0],
             [-1,-2,-3,-2,-1],
             [-2,-3,-4,-3,-2]]) / 23.0
    
    filter_array = np.zeros([5,5,1,2])
    filter_array[:,:,0,0] = filter0
    filter_array[:,:,0,1] = filter1

    return tf.constant(filter_array, dtype=tf.float32)

**[OFE-05]** 이미지 데이터에 필터를 적용하는 계산식을 준비한다.
x_image : 입력데이터. 이미지의 크기는 28x28이고, 레이어 개수는 1
pooling 후 14x14

In [0]:
x = tf.placeholder(tf.float32, [None, 784])
x_image = tf.reshape(x, [-1,28,28,1])

W_conv = edge_filter()
h_conv = tf.abs(tf.nn.conv2d(x_image, W_conv,
                             strides=[1,1,1,1], padding='SAME'))
h_conv_cutoff = tf.nn.relu(h_conv-0.2)

#h_pool =tf.nn.max_pool(h_conv_cutoff, ksize=[1,2,2,1],
#                       strides=[1,2,2,1], padding='SAME')


**[CNN-03]** 첫 번째 단계의 합성곱 필터와 풀링 계층을 정의한다.

In [0]:
num_filters1 = 64

W_conv1 = tf.Variable(tf.truncated_normal([5,5,2,num_filters1],
                                          stddev=0.1))
h_conv1 = tf.nn.conv2d(h_conv_cutoff, W_conv1,
                       strides=[1,1,1,1], padding='SAME')

b_conv1 = tf.Variable(tf.constant(0.1, shape=[num_filters1]))
h_conv1_cutoff = tf.nn.relu(h_conv1 + b_conv1)

h_pool1 = tf.nn.max_pool(h_conv1_cutoff, ksize=[1,2,2,1],
                         strides=[1,2,2,1], padding='SAME')

**[CNN-04]** 두 번째 단계의 합성곱 필터와 풀링 계층을 정의한다.

In [0]:
num_filters2 = 32

W_conv2 = tf.Variable(
            tf.truncated_normal([5,5,num_filters1,num_filters2],
                                stddev=0.1))
h_conv2 = tf.nn.conv2d(h_pool1, W_conv2,
                       strides=[1,1,1,1], padding='SAME')

b_conv2 = tf.Variable(tf.constant(0.1, shape=[num_filters2]))
h_conv2_cutoff = tf.nn.relu(h_conv2 + b_conv2)

h_pool2 = tf.nn.max_pool(h_conv2_cutoff, ksize=[1,2,2,1],
                         strides=[1,2,2,1], padding='SAME')

**[CNN-05]** 전 결합층, 드롭아웃 계층, 소프트맥스 함수를 정의한다.

In [0]:
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*num_filters2])

num_units1 = 7*7*num_filters2
num_units2 = 1024

w2 = tf.Variable(tf.truncated_normal([num_units1, num_units2]))
b2 = tf.Variable(tf.constant(0.1, shape=[num_units2]))
hidden2 = tf.nn.relu(tf.matmul(h_pool2_flat, w2) + b2)

keep_prob = tf.placeholder(tf.float32)
hidden2_drop = tf.nn.dropout(hidden2, keep_prob)

w0 = tf.Variable(tf.zeros([num_units2, 10]))
b0 = tf.Variable(tf.zeros([10]))
p = tf.nn.softmax(tf.matmul(hidden2_drop, w0) + b0)

**[CNN-06]** 오차 함수 loss, 트레이닝 알고리즘 train_step, 정답률 accuracy을 정의한다.

In [0]:
t = tf.placeholder(tf.float32, [None, 10])
loss = -tf.reduce_sum(t * tf.log(p))
train_step = tf.train.AdamOptimizer(0.0001).minimize(loss)
correct_prediction = tf.equal(tf.argmax(p, 1), tf.argmax(t, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

**[CNN-07]** 세션을 준비하고 Variable을 초기화한다.

In [0]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver()

/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


**[CNN-08]** 파라미터 최적화를 20000회 반복한다.

최종적으로 테스트 세트에 대해 약 99%의 정답률을 얻을 수 있다.

In [0]:
i = 0
start_vect=time.time()
for _ in range(20000):
    i += 1
    batch_xs, batch_ts = mnist.train.next_batch(50)
    sess.run(train_step,
             feed_dict={x:batch_xs, t:batch_ts, keep_prob:0.5})
    if i % 500 == 0:
        loss_vals, acc_vals = [], []
        for c in range(4):
            start = len(mnist.test.labels) // 4 * c
            end = len(mnist.test.labels) // 4 * (c+1)
            loss_val, acc_val = sess.run([loss, accuracy],
                feed_dict={x:mnist.test.images[start:end],
                           t:mnist.test.labels[start:end],
                           keep_prob:1.0})
            loss_vals.append(loss_val)
            acc_vals.append(acc_val)
        loss_val = np.sum(loss_vals)
        acc_val = np.mean(acc_vals)
        print (' %d,  %f, %f'
               % (i, loss_val, acc_val))
        saver.save(sess, './cnn_session', global_step=i)
print("training Runtime: %0.2f Minutes"%((time.time() - start_vect)/60))

 500,  2533.873535, 0.922900
 1000,  1828.822876, 0.943800
 1500,  1419.952515, 0.958100
 2000,  1215.441162, 0.961400
 2500,  1072.084961, 0.965100
 3000,  1027.372681, 0.966900
 3500,  882.474548, 0.970700
 4000,  851.102661, 0.972600
 4500,  797.994324, 0.975000
 5000,  834.413025, 0.971600
 5500,  724.399719, 0.976500
 6000,  713.550354, 0.976800
 6500,  637.223694, 0.979100
 7000,  697.147949, 0.977500
 7500,  642.250122, 0.977100
 8000,  684.412170, 0.978000
 8500,  560.065796, 0.981600
 9000,  565.792603, 0.980600
 9500,  666.611572, 0.978900
 10000,  563.484863, 0.982200
 10500,  577.681641, 0.982200
 11000,  570.229004, 0.981300
 11500,  560.126770, 0.981300
 12000,  541.362244, 0.981300
 12500,  531.623596, 0.983900
 13000,  491.855896, 0.984000
 13500,  495.611023, 0.983300
 14000,  509.355652, 0.983000
 14500,  487.979584, 0.983300
 15000,  473.656097, 0.983100
 15500,  500.640533, 0.983500
 16000,  478.444733, 0.984500
 16500,  469.383057, 0.984800
 17000,  502.013031, 0.9

**[CNN-09]** 세션 정보를 저장한 파일이 생성되고 있음을 확인한다.

In [0]:
!ls cnn_session*

cnn_session-18000.data-00000-of-00001  cnn_session-19000.meta
cnn_session-18000.index		       cnn_session-19500.data-00000-of-00001
cnn_session-18000.meta		       cnn_session-19500.index
cnn_session-18500.data-00000-of-00001  cnn_session-19500.meta
cnn_session-18500.index		       cnn_session-20000.data-00000-of-00001
cnn_session-18500.meta		       cnn_session-20000.index
cnn_session-19000.data-00000-of-00001  cnn_session-20000.meta
cnn_session-19000.index
